<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2cc446a6a6ae665cd1138cacaefbba3be99834a0e67d8c6e6f08ca9b37922283
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-30 09:26:56
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -72.69 K (-0.52%)
Current PnL: -27.36 L (-17.67%)
CY Booked + Current PnL: -13.07 L (-8.44%)
-------------------
Total profit:  1.42 L
Total loss:  -28.78 L
-------------------
Total Booked + Current PnL: 13.91 L (10.88%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.99 L (65.85%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.32,-19.78,24.77,0.09,20026.0,-19930.0,80847.0,87.62,37.0,M-SC,3.03,253.0,-1.00,0.58,6.21,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.59,7.33,11.52,19.69,23708.0,14058.0,205800.0,-3.15,49.0,X-MC,2.30,78.0,0.59,1.48,14.63,XY25,NTT,AC
52,NESTLEIND,1377.00,-0.46,12.32,8.08,21.40,24090.0,32713.0,298139.0,4.17,61.0,X-LC,5.29,12.0,1.36,2.14,19.43,XY25,NTT,FMCG
50,MASFIN,398.61,-0.32,-4.61,27.94,22.05,26114.0,-4515.0,93465.0,-17.80,46.0,H-SC,6.47,164.0,-0.17,0.67,35.86,XR,ATH,FINANCE
43,ITC,452.00,-0.15,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,2.00,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,0.79,-17.93,79.06,46.95,81297.0,-22468.0,102829.0,-24.08,40.0,H-SC,10.24,154.0,-0.28,0.74,28.82,MH,ATH,POWER
75,TMPV,600.00,0.52,-26.12,66.46,22.98,110914.0,-59010.0,166888.0,-23.08,51.0,X-LC,6.17,3.0,-0.53,1.20,4.97,XY24,NTT,AUTO
36,ICICIPRULI,790.00,0.36,8.15,20.88,30.73,44486.0,16052.0,213057.0,-17.13,63.0,X-MC,3.88,75.0,0.36,1.53,21.96,X40,ATH,INSURANCE
23,DMART,5391.45,0.23,-8.73,42.00,29.60,95679.0,-21792.0,227808.0,-22.18,37.0,X-LC,8.24,19.0,-0.23,1.64,11.54,X40N,ATH,FMCG
20,DABUR,735.00,0.23,-4.34,50.21,43.70,119153.0,-10758.0,237310.0,-12.17,33.0,X-MC,2.00,72.0,-0.09,1.71,10.06,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-5.00,-62.90,169.72,0.07,86376.0,-86284.0,50893.0,1764.90,46.0,L-SC,4.55,268.0,-1.00,0.37,26.57,OX40N,NTT,JEWELLERY
44,JCHAC,2282.0,-4.24,-37.58,60.20,-0.01,48023.0,-48033.0,79772.0,16226.09,35.0,M-SC,13.03,233.0,-1.00,0.57,2.10,OX40N,NTT,AC
13,BSOFT,831.7,-2.72,-20.85,90.52,50.79,100767.0,-29333.0,111320.0,1.66,58.0,H-SC,10.78,151.0,-0.29,0.80,29.61,XR,ATH,IT
0,5PAISA,593.0,-1.87,-36.67,78.27,12.90,108312.0,-80114.0,138382.0,104.33,57.0,H-SC,12.31,161.0,-0.74,1.00,15.06,OX40N,NTT,FINANCE
74,TITAGARH,1548.0,-1.73,-16.03,73.56,45.74,149586.0,-38824.0,203353.0,11.88,66.0,H-SC,5.25,174.0,-0.26,1.46,32.26,XY24,NTT,ENGINEERING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.97,-6.91,123.34,107.91,169525.0,-10199.0,137445.0,-24.20,38.0,M-SC,11.61,216.0,-0.06,0.99,2.48,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,-0.32,-19.78,24.77,0.09,20026.0,-19930.0,80847.0,87.62,37.0,M-SC,3.03,253.0,-1.00,0.58,6.21,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.38,-20.25,25.42,0.02,35385.0,-35357.0,139202.0,112.05,33.0,M-SC,8.98,234.0,-1.00,1.00,21.08,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-1.27,-22.74,46.87,13.47,97654.0,-61317.0,208350.0,-68.31,50.0,H-SC,6.84,158.0,-0.63,1.50,5.94,XY24,NTT,PAINTS
66,SIS,528.00,-0.82,-23.42,58.34,21.26,49607.0,-26002.0,85030.0,2006.74,52.0,H-SC,4.60,166.0,-0.52,0.61,14.88,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.18,8.19,59.23,72.27,108580.0,13872.0,183320.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.32,14.87,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.18,8.19,59.23,72.27,108580.0,13872.0,183320.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.32,14.87,XR,NTT,IT
6,ASIANTILES,137.00,-0.56,-1.45,84.34,81.67,77976.0,-1356.0,92454.0,8327.78,65.0,L-SC,15.88,271.0,-0.02,0.67,79.04,XR,NTT,CERAMICS
38,INDIAMART,4810.62,-0.25,-5.72,119.27,106.72,138684.0,-7059.0,116277.0,-53.30,36.0,H-SC,5.42,139.0,-0.05,0.84,16.32,AR,ATH,MISC
25,FINCABLES,1641.55,-0.97,-6.91,123.34,107.91,169525.0,-10199.0,137445.0,-24.20,38.0,M-SC,11.61,216.0,-0.06,0.99,2.48,OX40N,ATH,CABLES
1,ABB,7934.00,-0.33,-5.55,54.12,45.57,133733.0,-14515.0,247104.0,-39.35,48.0,H-MC,4.57,120.0,-0.11,1.78,5.14,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,0.08,-20.98,59.53,26.07,91838.0,-40950.0,154271.0,-61.61,24.0,X-MC,15.45,56.0,-0.45,1.11,0.08,X40N,ATH,IT
9,BAJAJHFL,181.50,-0.17,-19.47,93.06,55.47,182320.0,-47369.0,195917.0,-28.68,26.0,X-MC,17.00,64.0,-0.26,1.41,0.38,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.32,-19.24,77.46,43.32,101891.0,-31332.0,131540.0,-30.65,28.0,X-SC,6.94,90.0,-0.31,0.95,1.00,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.47,-27.84,127.48,64.15,218896.0,-66241.0,171710.0,-57.62,29.0,X-SC,5.41,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT
8,AWL,485.00,-0.48,-25.20,105.20,53.50,237451.0,-76037.0,225714.0,-63.76,31.0,X-MC,7.17,58.0,-0.32,1.62,0.25,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-1.05,-4.00,26.18,21.14,57139.0,-9085.0,218254.0,-10.72,43.0,X-MC,1.35,74.0,-0.16,1.57,21.20,XY24,NTT,PAINTS
65,SIEMENS,4671.5,-0.57,-18.40,53.82,25.51,81726.0,-34245.0,151850.0,-6.88,36.0,H-LC,1.47,49.0,-0.42,1.09,11.90,AR,ATH,ELECTRICAL
78,UNITDSPR,1644.0,-0.17,4.85,15.18,20.77,37266.0,11352.0,245496.0,-4.29,50.0,X-MC,1.66,62.0,0.30,1.77,12.10,X40N,NTT,BREWERIES
43,ITC,452.0,-0.15,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,2.00,5.0,-0.12,1.70,3.69,X40,NTT,FMCG
20,DABUR,735.0,0.23,-4.34,50.21,43.70,119153.0,-10758.0,237310.0,-12.17,33.0,X-MC,2.00,72.0,-0.09,1.71,10.06,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.47,-27.84,127.48,64.15,218896.0,-66241.0,171710.0,-57.62,29.0,X-SC,5.41,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.41,-39.56,123.69,35.20,96195.0,-50899.0,77771.0,1.95,32.0,X-SC,17.35,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.30,-21.64,80.58,41.51,166301.0,-56985.0,206380.0,-6.75,32.0,X-MC,8.38,61.0,-0.34,1.48,0.00,XY25,ATH,FMCG
22,DIXON,18931.72,0.08,-20.98,59.53,26.07,91838.0,-40950.0,154271.0,-61.61,24.0,X-MC,15.45,56.0,-0.45,1.11,0.08,X40N,ATH,IT
8,AWL,485.00,-0.48,-25.20,105.20,53.50,237451.0,-76037.0,225714.0,-63.76,31.0,X-MC,7.17,58.0,-0.32,1.62,0.25,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.80,-29.69,103.15,42.84,47147.0,-19299.0,45707.0,-53.72,44.0,X-SC,32.08,83.0,-0.41,0.33,3.41,XY24,NTT,MISC
59,RELAXO,1176.00,-0.62,-48.09,193.41,52.31,145734.0,-69810.0,75350.0,-44.73,40.0,X-SC,7.36,91.0,-0.48,0.54,0.86,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.41,-39.56,123.69,35.20,96195.0,-50899.0,77771.0,1.95,32.0,X-SC,17.35,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.49,-4.16,29.47,24.08,35855.0,-5283.0,121667.0,-11.91,38.0,X-SC,7.50,89.0,-0.15,0.88,14.64,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.32,-19.24,77.46,43.32,101891.0,-31332.0,131540.0,-30.65,28.0,X-SC,6.94,90.0,-0.31,0.95,1.00,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.23,-10.80,35.32,20.71,119165.0,-40851.0,337386.0,-23.30,54.0,X-LC,3.09,1.0,-0.34,2.43,12.73,X40,ATH,IT
41,INFY,1972.00,-0.35,11.57,20.32,34.24,70934.0,36198.0,349086.0,-13.14,59.0,X-LC,7.32,2.0,0.51,2.51,20.81,X40,NTT,IT
75,TMPV,600.00,0.52,-26.12,66.46,22.98,110914.0,-59010.0,166888.0,-23.08,51.0,X-LC,6.17,3.0,-0.53,1.20,4.97,XY24,NTT,AUTO
81,VBL,671.64,-0.15,-2.64,39.34,35.67,120977.0,-8326.0,307516.0,-14.21,56.0,X-LC,3.04,4.0,-0.07,2.21,10.85,X40N,ATH,FMCG
43,ITC,452.00,-0.15,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,2.00,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.10,-37.95,114.18,32.89,54696.0,-29304.0,47903.0,-691.63,50.0,L-MC,6.70,259.0,-0.54,0.34,31.92,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.56,-1.45,84.34,81.67,77976.0,-1356.0,92454.0,8327.78,65.0,L-SC,15.88,271.0,-0.02,0.67,79.04,XR,NTT,CERAMICS
13,BSOFT,831.70,-2.72,-20.85,90.52,50.79,100767.0,-29333.0,111320.0,1.66,58.0,H-SC,10.78,151.0,-0.29,0.80,29.61,XR,ATH,IT
17,COALINDIA,484.83,-0.57,-2.04,21.71,19.23,31998.0,-3070.0,147390.0,15.50,66.0,L-LC,3.40,179.0,-0.10,1.06,18.80,XY25,ATH,MINING
84,WIPRO,420.00,-0.18,8.19,59.23,72.27,108580.0,13872.0,183320.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.32,14.87,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-0.56,-1.45,84.34,81.67,77976.0,-1356.0,92454.0,8327.78,65.0,L-SC,15.88,271.0,-0.02,0.67,79.04,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.72,-20.85,90.52,50.79,100767.0,-29333.0,111320.0,1.66,58.0,H-SC,10.78,151.0,-0.29,0.80,29.61,XR,ATH,IT
74,TITAGARH,1548.0,-1.73,-16.03,73.56,45.74,149586.0,-38824.0,203353.0,11.88,66.0,H-SC,5.25,174.0,-0.26,1.46,32.26,XY24,NTT,ENGINEERING
84,WIPRO,420.0,-0.18,8.19,59.23,72.27,108580.0,13872.0,183320.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.32,14.87,XR,NTT,IT
36,ICICIPRULI,790.0,0.36,8.15,20.88,30.73,44486.0,16052.0,213057.0,-17.13,63.0,X-MC,3.88,75.0,0.36,1.53,21.96,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.41
1,25,45.05
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.85
MC    30.84
LC    24.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.89
X40      23.16
X40N     12.91
XR        9.82
XY25      9.21
AR        8.82
OX40N     7.49
SR        0.99
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.27
H-SC    23.83
X-LC    20.87
M-SC    11.43
X-SC     8.04
H-MC     4.83
L-SC     1.55
M-MC     1.40
M-LC     1.32
H-LC     1.09
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.44,-7.59,42.18
IT,12.91,-17.06,75.92
FINANCE,9.64,-18.87,70.56
MISC,7.17,-30.56,83.97
ELECTRICAL,5.95,-13.55,55.15
PAINTS,5.82,-14.13,31.08
INSURANCE,4.74,-2.92,38.14
PHARMA,4.08,-3.79,36.46
AUTO,2.85,-32.07,77.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3182896.0,21
AR,1346787.0,10
XR,1297895.0,13
X40,1004776.0,14
X40N,892347.0,9
OX40N,750649.0,10
XY25,359473.0,6
SR,283014.0,2
MH,81297.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3666949.0,25
M-SC,1467633.0,15
X-MC,1453841.0,16
X-LC,824450.0,11
X-SC,804449.0,8
H-MC,408160.0,3
L-SC,246622.0,3
M-LC,108580.0,1
H-LC,81726.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1258257.0      6
           AR         917533.0      5
M-SC       XY24       827778.0      6
H-SC       XR         795430.0      7
X-MC       X40        492799.0      7
           XY24       413743.0      3
           X40N       357290.0      4
M-SC       OX40N      332855.0      5
H-SC       OX40N      331418.0      4
X-LC       X40        327827.0      5
X-SC       X40N       318401.0      3
           XY24       301898.0      3
H-SC       SR         283014.0      2
H-MC       AR         219471.0      2
X-LC       X40N       216656.0      2
           XY24       192531.0      2
X-MC       XY25       190009.0      2
H-MC       XY24       188689.0      1
X-SC       X40        184150.0      2
M-SC       XR         178943.0      2
L-SC       XR         160246.0      2
M-SC       AR         128057.0      2
M-LC       XR         108580.0      1
X-LC       XY25        87436.0      2
L-SC       OX40N       86376.0      1
H-LC       AR          81726.0      1
H-SC       MH          81297.0      1
L-MC       XR          54696.0      1
M-MC       XY25        50030.0      1
L-LC       XY25        31998.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
